In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("non_aggregated_counts.csv")

In [ ]:
def get_stats(data, confidence_level=0.95):
    data_mean = np.mean(data)
    data_std_dev = np.std(data)
    # Calculate the critical value based on the confidence level and a normal distribution
    critical_value = stats.norm.ppf((1 + confidence_level) / 2)

    # Calculate the standard error of the mean for a population
    standard_error = data_std_dev / np.sqrt(len(data))

    # Calculate the margin of error
    margin_of_error = critical_value * standard_error

    # Calculate the confidence interval
    confidence_interval = (data_mean - margin_of_error, data_mean + margin_of_error)
    return (confidence_interval, margin_of_error, data_mean, data_std_dev, critical_value, standard_error)

In [ ]:
#df_grouped = df.groupby(['source', 'country', 'dimension'])['count'].sum()
df['count_state_source_dim'] = df.groupby(['dimension', 'country', 'source'])['count'].transform('sum')

dims = list(set(pd.Series(df['dimension'])))
# Calculate mean and confidence intervals
mean_ci = df.groupby(['dimension', 'source']).agg(
    mean_count=('count_state_source_dim', 'mean'),
    ci=('count_state_source_dim', lambda x: x.sem() * 1.96)  # 95% confidence interval
).reset_index()
collapsed_df = df.groupby(['dimension', 'source'], as_index=False)['count_state_source_dim'].agg(['mean', 'count', 'std'])
ci = []
me = []
for i in collapsed_df.index:
    m, c, s = collapsed_df.loc[i]    
    ci.append( ((m + 1.96*s/math.sqrt(c)), (m - 1.96*s/math.sqrt(c))) )
    me.append(1.96*s/math.sqrt(c))
 
# Collapse the data to get the first value of 'count_state_source_dim' per group
collapsed_df = df.groupby(['dimension', 'source'], as_index=False).agg({'count_state_source_dim': 'mean'})
collapsed_df['margin_error'] = me
# Calculate confidence intervals
#confidence_intervals = df.groupby(['dimension', 'source'], as_index=False).agg({'count_state_source_dim': lambda x: x.sem() * 1.96})

In [ ]:
# Encode 'dimension' and 'source' columns to create new numeric columns
#collapsed_df['dimension_new'] = pd.factorize(collapsed_df['dimension'])[0]
#collapsed_df['source_new'] = pd.factorize(collapsed_df['source'])[0]
#print(collapsed_df)
print(collapsed_df)

In [ ]:
def plot_sources(df, title=None):    
    
    fig = go.Figure()
    dims = list(set(pd.Series(df['dimension'])))
    for dim in dims:
        f_df = df[df['dimension'] == dim]        
        fig.add_trace(go.Bar(x=f_df['source'], 
                             y=f_df['count_state_source_dim'], 
                             error_y=dict(type='data', array=f_df['margin_error'], visible=True), 
                             name=dim))
 

    # Iterate over each column (excluding the 'Category' column)
    #for col in df.columns[2:]:
    #    fig.add_trace(go.Bar(x=df[x], y=df[col], name=get_topic_name(col)))

    # Customize the layout (optional)
    #fig.update_layout(title=title, xaxis_title=x, yaxis_title='counts')

    # Display the chart
    #fig.show()
    return fig

In [ ]:
plot_sources(collapsed_df)